In [10]:
from cvat_sdk import make_client
import os
from cvat_sdk.api_client import Configuration, ApiClient, exceptions
from cvat_sdk.api_client.models import DataRequest, StorageMethod, StorageType, SortingMethod

# def teste_inserir_imagem(cvat_host: str, username: str, password: str, task_id: int, image_paths: list):

#     try:
#         # Cria um cliente CVAT autenticado
#         with make_client(host=cvat_host) as client:
#             client.login((username, password))



#             task = client.tasks.retrieve(obj_id=task_id)

#             # Verifica se todos os caminhos das imagens são válidos
#             for image_path in image_paths:
#                 if not os.path.isfile(image_path):
#                     raise FileNotFoundError(f"A imagem '{image_path}' não foi encontrada.")

#             # Cria a task no CVAT
   
#             task.import_annotations()
            
            
#             created_date.(
#                 spec=task_spec, 
#                 #resource_type="local",  # Indicamos que os arquivos de imagem são locais
#                 resources=image_paths, # Passamos a lista de caminhos para as imagens
#                 data_params={"quality":100}
#             )




#             print(f"Task '{task.name}' criada com sucesso! ID da Task: {task.id}")
#             return {
#                 "ID": task.id,
#                 "Nome": task.name,
#                 "Projeto": task.project_id,
#                 "Status": task.status
#             }

#     except Exception as e:
#         print(f"Erro ao criar a task: {e}")
#         return {"erro": str(e)}



def criar_task_sem_imagens(cvat_host: str, username: str, password: str, project_id: int, task_name: str):
    """
    Cria uma task no CVAT sem associar nenhuma imagem.

    Parâmetros:
    - cvat_host (str): URL do servidor CVAT.
    - username (str): Nome de usuário para autenticação.
    - password (str): Senha para autenticação.
    - project_id (int): ID do projeto onde a task será criada.
    - task_name (str): Nome da task a ser criada.

    Retorna:
    - dict: Detalhes da task criada, ou mensagem de erro.
    """
    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_host) as client:
            client.login((username, password))

            # Prepara o payload para a criação da task sem imagens
            task_spec = {
                "name": task_name,
                "project_id": project_id,
            }

            # Cria a task no CVAT
            task = client.tasks.create(task_spec)

            print(f"Task '{task.name}' criada com sucesso! ID da Task: {task.id}")
            return {
                "ID": task.id,
                "Nome": task.name,
                "Projeto": task.project_id,
                "Status": task.status
            }

    except Exception as e:
        print(f"Erro ao criar a task: {e}")
        return {"erro": str(e)}

def listar_projetos(cvat_url: str, username: str, password: str):
    """
    Lista todos os projetos do CVAT.

    Parâmetros:
    cvat_host (str): URL do servidor CVAT.
    username (str): Nome de usuário para autenticação.
    password (str): Senha para autenticação.

    Retorna:
    list: Uma lista de dicionários contendo os detalhes dos projetos.
    """
    projetos = []
    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_url) as client:
            client.login((username, password))

            # Lista todos os projetos
            projects = client.projects.list()

            # Adiciona os detalhes dos projetos à lista de retorno
            for project in projects:
                projetos.append({
                    "ID": project.id,
                    "Nome": project.name,
                    "Status": project.status
                })

    except Exception as e:
        print(f"Erro ao listar projetos: {e}")

    return projetos

   
def criar_task_com_imagens(cvat_host: str, username: str, password: str, project_id: int, task_name: str, image_paths: list, slug_organization = None):

    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_host) as client:
            client.login((username, password))

            # Prepara o payload para a criação da task
            task_spec = {
                "name": task_name,
                "project_id": project_id,
            }
            
            if slug_organization != None:
                client.organization_context(slug=slug_organization)

            # Verifica se todos os caminhos das imagens são válidos
            for image_path in image_paths:
                if not os.path.isfile(image_path):
                    raise FileNotFoundError(f"A imagem '{image_path}' não foi encontrada.")

            # Cria a task no CVAT
            task = client.tasks.create_from_data(
                spec=task_spec, 
                #resource_type="local",  # Indicamos que os arquivos de imagem são locais
                resources=image_paths, # Passamos a lista de caminhos para as imagens
                data_params={"quality":100}
            )

            print(f"Task '{task.name}' criada com sucesso! ID da Task: {task.id}")
            return {
                "ID": task.id,
                "Nome": task.name,
                "Projeto": task.project_id,
                "Status": task.status
            }

    except Exception as e:
        print(f"Erro ao criar a task: {e}")
        return {"erro": str(e)}


def adicionar_imagens_a_task(cvat_host: str, username: str, password: str, task_id: int, image_paths: list):
    """
    Adiciona imagens a uma tarefa existente no CVAT.

    Parâmetros:
    cvat_host (str): URL do servidor CVAT.
    username (str): Nome de usuário para autenticação.
    password (str): Senha para autenticação.
    task_id (int): ID da tarefa existente.
    image_paths (list): Lista de caminhos para os arquivos de imagem que serão carregados.

    Retorna:
    dict: Detalhes do upload, ou mensagem de erro.
    """
    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_host) as client:
            client.login((username, password))

            # Verifica se todos os caminhos das imagens são válidos
            for image_path in image_paths:
                if not os.path.isfile(image_path):
                    raise FileNotFoundError(f"A imagem '{image_path}' não foi encontrada.")

            # Abre os arquivos de imagem para upload
            with open_images(image_paths) as image_files:
                # Faz o upload das imagens para a task existente
                client.tasks.create_data(task_id=task_id, resources=image_files, data_params={"quality": 100})

            print(f"Imagens adicionadas com sucesso à tarefa ID: {task_id}")

    except Exception as e:
        print(f"Erro ao adicionar imagens à tarefa: {e}")
        return {"erro": str(e)}


def adicionar_bounding_boxes(cvat_host: str, username: str, password: str, task_id: int, annotations: list):
    """
    Adiciona bounding boxes (caixas delimitadoras) com rótulos (labels) às imagens de uma tarefa no CVAT.

    Parâmetros:
    cvat_host (str): URL do servidor CVAT.
    username (str): Nome de usuário para autenticação.
    password (str): Senha para autenticação.
    task_id (int): ID da tarefa onde as anotações serão adicionadas.
    annotations (list): Lista de dicionários com anotações de bounding boxes.

    Exemplo de `annotations`:
    [
        {
            "frame": 0,
            "label_id": 1,  # ID do rótulo (defeito)
            "xtl": 100,  # Coordenada X do canto superior esquerdo da caixa
            "ytl": 150,  # Coordenada Y do canto superior esquerdo da caixa
            "xbr": 200,  # Coordenada X do canto inferior direito da caixa
            "ybr": 250   # Coordenada Y do canto inferior direito da caixa
        },
        ...
    ]
    """
    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_host) as client:
            client.login((username, password))
            
            # Obter a tarefa
            task = client.tasks.retrieve(task_id)

            # Preparar as anotações (bounding boxes)
            labeled_shapes = []
            for annotation in annotations:
                shape = LabeledShapeRequest(
                    frame=annotation["frame"],  # O frame (imagem) em que a caixa será inserida
                    label_id=annotation["label_id"],  # ID do rótulo (defeito)
                    type="rectangle",  # Tipo de forma (bounding box é um retângulo)
                    points=[annotation["xtl"], annotation["ytl"], annotation["xbr"], annotation["ybr"]],  # Coordenadas da caixa
                    occluded=False,  # Não está oculto
                    z_order=0,  # Ordem Z (caso haja sobreposição de caixas)
                    group=0,  # Agrupamento de caixas (não usado neste exemplo)
                    attributes=[]  # Atributos adicionais (se necessário)
                )
                labeled_shapes.append(shape)

            # Adicionar as anotações à tarefa
            client.api.tasks.create_annotations(task_id=task_id, shapes=labeled_shapes)

            print(f"Anotações adicionadas com sucesso à tarefa ID: {task_id}")

    except Exception as e:
        print(f"Erro ao adicionar anotações: {e}")


def listar_tasks(cvat_host: str, username: str, password: str):
    """
    Lista todas as tasks do CVAT.

    Parâmetros:
    cvat_host (str): URL do servidor CVAT.
    username (str): Nome de usuário para autenticação.
    password (str): Senha para autenticação.

    Retorna:
    list: Uma lista de dicionários contendo os detalhes das tasks.
    """
    tasks_list = []
    try:
        # Cria um cliente CVAT autenticado
        with make_client(host=cvat_host) as client:
            client.login((username, password))

            # Lista todas as tasks
            tasks = client.tasks.list()

            # Adiciona os detalhes das tasks à lista de retorno
            for task in tasks:
                tasks_list.append({
                    "ID": task.id,
                    "Nome": task.name,
                    "Status": task.status,
                    "Projeto": task.project_id,
                    "Data de Criação": task.created_date
                })

    except Exception as e:
        print(f"Erro ao listar tasks: {e}")

    return tasks_list


def open_images(image_paths):
    """
    Gerenciador de contexto para abrir múltiplos arquivos de imagem.
    """
    image_files = [open(image_path, 'rb') for image_path in image_paths]
    try:
        yield image_files
    finally:
        for image_file in image_files:
            image_file.close()

In [15]:
# Exemplo de uso da função
cvat_url = "http://10.247.141.99:8080"
username = "igor.santos@aperam.com"
password = "qwer1234Q."

listar_projetos(cvat_url, username, password)

[{'ID': 20, 'Nome': 'Primeiro projeto', 'Status': 'annotation'},
 {'ID': 19, 'Nome': 'Emissoes fugitivas', 'Status': 'annotation'},
 {'ID': 18, 'Nome': 'Argus - RB3 - Automate', 'Status': 'annotation'},
 {'ID': 16, 'Nome': 'Automate insert image Argus', 'Status': 'annotation'},
 {'ID': 15, 'Nome': 'detecçãoPelotas', 'Status': 'annotation'},
 {'ID': 14, 'Nome': 'ARGUS - RB3', 'Status': 'annotation'},
 {'ID': 13, 'Nome': 'Argus', 'Status': 'annotation'},
 {'ID': 2,
  'Nome': 'Defect Inspection Project - Argus',
  'Status': 'annotation'}]

In [17]:
from datetime import date

data_atual = date.today()

project_id = 20  # Substitua pelo ID real do seu projeto
task_name = f"New task date 123: {data_atual}"
image_paths = ['/home/igor/projects/CVAT/saved_images/2024_10_17_16_17_20_1.png','/home/igor/projects/CVAT/saved_images/2024_10_17_16_17_20_1.png' ]
# Criando a task com upload das imagens

task_info = criar_task_com_imagens(cvat_url, username, password, project_id, task_name, image_paths, "Igor")


Erro ao criar a task: Status Code: 400
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Allow': 'GET, POST, HEAD, OPTIONS', 'Content-Length': '64', 'Content-Type': 'application/vnd.cvat+json', 'Cross-Origin-Opener-Policy': 'same-origin', 'Date': 'Thu, 17 Oct 2024 20:18:24 GMT', 'Referrer-Policy': 'same-origin, strict-origin-when-cross-origin', 'Server': 'nginx', 'Vary': 'Accept, Origin', 'X-Content-Type-Options': 'nosniff, nosniff', 'X-Frame-Options': 'DENY', 'X-Request-Id': '9d6c2f31-e2d2-4758-9b44-c8d630216032'})
HTTP response body: ["The task and its project should be in the same organization."]



In [13]:

cvat_url = "http://10.247.141.99:8080"
username = "igor.santos@aperam.com"
password = "qwer1234Q."
project_id = 16  # Substitua pelo ID real do seu projeto
task_name = "Task com defeito fictício"
image_paths = []  # Substitua com o caminho real da sua imagem

# Criando a task e obtendo o ID
task_id = criar_task_sem_imagens(cvat_url, username, password, project_id, task_name, "MA")




TypeError: criar_task_sem_imagens() takes 5 positional arguments but 6 were given

In [ ]:
image_paths = ["/home/igor/projects/CVAT/2_fast_process_module/app/jupyter_notebook/2024_10_16_17_00_53_1.png" ]

enviar_imagens_para_task(cvat_url, username, password, 63, image_paths)